# Web Scraping usando Selenium

#### Código en python que realiza los siguientes pasos:

* Abre el navegador web

* Ingresa a la pagina web https://www.adamchoi.co.uk/teamgoals/detailed
* Selecciona el boton 'All matches'
* Selecciona el país 'Spain'
* Extrae la información de todos los partidos
* Cierra el navegador web
* Pasa todo a dataframe
* Presenta la información por pantalla
* Crea la carpeta 'files' para guardar el archivo 'partidos.csv'

In [87]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
import os

##### --> Abrir el navegador web e ingresar a la página

In [79]:
# Establecer la pagina web a trabajar
website = 'https://www.adamchoi.co.uk/teamgoals/detailed'

# Estblecer la ruta del archibo ChromeDriver descargado
path = '/Users/mac/Desktop/Web_Scraping/2-Selenium/ChromeDriver/chromedriver'

*** IMPORTANTE ***

* Si al ejecutar el código del box de abajo devuelve el error: "La app chromedriver no se puede abrir porque el desarrollador no puede verificarse. No se puede verificar que esta app no contiene software malicioso"

* En Configuración del Sistema ir a Ajustes de sistema / Privacidad y seguridad / Seguridad / se verá la leyenda "Se ha bloqueado el uso de chromedriver..." --> "Permitir de todos modos"

* Volver al VSC y volver a correr el codigo

In [80]:
# Definir la variable que abrira la web
driver = webdriver.Chrome(path)

# Dar la indicación para que abra la web
driver.get(website)

/var/folders/kg/01lphnns7wq7wxsptrz_jyc00000gq/T/ipykernel_58699/2213360679.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


##### --> Seleccionar el botón que dice "All Matches"

In [36]:
# --> Método 1: usando Xpath ABSOLUTO

# Ingresar al código html de la página, mediante la opción 'Inspect' y usar "Select an element in the page to inspect it"
# Ubicar el código del botón 'All matches'
# Click derecho, Copy, Xpath
# Pegarlo como value en el comando 'driver.find_element'

'''
all_matches_button = driver.find_element(by='xpath', value='//*[@id="page-wrapper"]/div/home-away-selector/div/div/div/div/label[2]')
all_matches_button.click()
'''

# Si bien esto funciona, no es la manera conveniente porque al ser tan extenso, contiene muchas dependencias,
# lo que implica que ante un mínimo cambio en la estructura de la web, devolverá error.

'\nall_matches_button = driver.find_element(by=\'xpath\', value=\'//*[@id="page-wrapper"]/div/home-away-selector/div/div/div/div/label[2]\')\nall_matches_button.click()\n'

In [84]:
# --> Método 2: usando Xpath RELATIVO

# Entonces, del mismo cödigo del botón identificado arriba, se edita el xpath absoluto:

all_matches_button = driver.find_element(by='xpath', value='//label[@analytics-event="All matches"]') 
all_matches_button.click()

# Como opción, se podría utilizar la clase como parámetro, pero no se utilioza porque resulta también 
# extensa y porque existe la alternativa que se aplicó en el código. Pero era otra posibilidad.

# La instrucción se leería como:
# Buscame un tag 'xpath' que tenga un atributo 'analytics-event' con un texto 'All matches'.

#### --> Seleccionar el país 'Spain' (dentro del menú desplegable 'Select country')

In [85]:
# Luego de ubicar el código que corresponde al desplegable, seguir el orden de aplicación, siendo lo
# primero buscar si dentro existe un id (porque son únicos). Y existe!
# En este caso, al ser un desplegable y no un botón, el manejo es diferente ya que no se puede ver el 
# código puntual de 'Spain', sino que se accede sólo al del desplegable.
# Para estos casos se necesita la librería Select, perteneciente a Selenium.

dropdown = Select(driver.find_element(by='id', value='country'))
dropdown.select_by_visible_text('Spain')

In [ ]:
# Otra opción hubiera sido:

# Para pasar una referencia sobre la porción del código con la que queremos trabajar, se puede tomar un
# identificador de alguno de los nodos padres del elemento que necesitamos usar. Esto porque a veces no
# hay un identificador que sea único, por lo que es útil usar alguna referencia de los nodos padres y
# acotar el sector del código que se pasa como parámetro.

'''
caja = driver.find_element(by='class_name', value='panel-body')

# Y luego usar:
dropdown = Select(caja.find_element(by='id', value='country'))
'''


In [88]:
# Establecer 5 segundos de espera por el tiempo de carga que tiene la página
time.sleep(5)

##### --> Extracción de los datos de todos los partidos (desde la caja de los matches)

In [89]:
# Lo primero es identificar, en el código, un patrón común que permita agrupar los datos a extraer ('tr').
# Ahora el 'find_elements' va con la 's' al final porque aca el tag 'tr' no es unico, sino que hay muchos.

matches = driver.find_elements(by='xpath', value='//tr')

In [90]:
# En matches ya están cargados todos los partidos. Ahora va entrando en cada uno para mandarlos a una lista.

partidos = []
for match in matches:
    #print(match.text)
    partidos.append(match.text)

##### --> Cerrar el navegador web

In [91]:
# Se cierra el driver (que se habia abierto con el 'get').
driver.quit()

##### --> Pasar los datos a dataframe, mostrar por pantalla y guardar en disco

In [92]:
# Mediante pandas, se convierten los datos a dataframe.
df = pd.DataFrame({'Partidos de La Liga' : partidos})

# Mostrar por pantalla.
print(df)

# Crear la carpeta para guardar el archivo, si no existe.
path = "./Files"
if not os.path.exists(path):
    os.makedirs(path)

# Guardar los datos en archivo.
df.to_csv('./Files/partidos.csv', index=False)

                        Partidos de La Liga
0      14-08-2022 Almeria 1 - 2 Real Madrid
1            22-08-2022 Elche 1 - 1 Almeria
2          27-08-2022 Almeria 2 - 1 Sevilla
3       05-09-2022 Valladolid 1 - 0 Almeria
4          12-09-2022 Almeria 0 - 1 Osasuna
..                                      ...
695     27-04-2023 Villarreal 4 - 2 Espanol
696       30-04-2023 Villarreal 3 - 1 Celta
697    03-05-2023 Valencia 1 - 1 Villarreal
698  13-05-2023 Villarreal 5 - 1 Ath Bilbao
699      20-05-2023 Girona ? - ? Villarreal

[700 rows x 1 columns]
